# River tracers

A visualisation of passive river tracers for the Don, O'Connell, and Pioneer rivers near the Whitsundays.
Passive river tracer results derived from version 2.0 of the 1km-resolution shelf-scale hydrodynamic model of the Great Barrier Reef (GBR1).

In [ ]:
import cartopy
import emsarray
import matplotlib.colors
import matplotlib.figure
import matplotlib.pyplot as plt
import matplotlib.quiver
import numpy
import pandas
import shapely
import xarray
from matplotlib.offsetbox import (
    AnnotationBbox, DrawingArea, HPacker, TextArea, PaddedBox,
)

# width, height in pixels
IMAGE_SIZE = (1000, 1000)
DPI = 100 # matplotlib default

# The plot will cover this region, expanding as required to match the aspect ratio set in IMAGE_SIZE.
# The coordinates represent the (west, east, south, north) sides of a region that will be plotted.
MAP_EXTENT = (147.80, 149.60, -21.35, -19.55)

RIVERS_URL = 'https://thredds.nci.org.au/thredds/dodsC/fx3/model_data/gbr1_2.0_rivers.ncml'
RIVERS_TIMESTEP = '2022-03-01T14:00:00.00000000'

MODIS_TIMESTEP = '2022-03-01'

Open the GBR1 v2.0 rivers dataset and select one time step at the ocean surface. Select only the rivers we are interested in and then fetch that data in to memory.

In [ ]:
rivers_dataset = emsarray.open_dataset(RIVERS_URL, decode_timedelta=False)

rivers_dataset = rivers_dataset.set_coords(['time', 'zc'])
rivers_dataset = rivers_dataset.set_xindex(['zc'])
rivers_dataset = rivers_dataset.sel(time=RIVERS_TIMESTEP, zc=0.5)

# Collect the Don, O'Connell, and Pioneer river tracers
river_names = [
    'don', # Don River
    'con', # O'Connell River
    'pio', # Pioneer River
]

# Extract the river variables we need
rivers_dataset = rivers_dataset.ems.select_variables(river_names)
rivers_dataset.load()
convention = rivers_dataset.ems

Exract some useful facts about the rivers

In [ ]:
river_labels = {
    river_name: rivers_dataset[river_name].attrs['long_name']
    for river_name in river_names
}

# Where the rivers enter the ocean, to attach a label on the map
river_coords = {
    'don': (148.22461059096432, -19.963993553556246),
    'con': (148.66747821623494, -20.572470925022575),
    'pio': (149.21411312349773, -21.141118013698640),
}


# Make a DataArray with each of the rivers stacked along a new dimension 'tracer'.
river_tracers = xarray.concat([
    rivers_dataset[name].assign_coords({'tracer': name})
    for name in river_names
], dim='tracer')
river_tracers.name = 'tracers'

# Set the maximum river concentration to visualise.
# The tracer dissipates quickly, setting this somewhere in the range of
# 5%-10% has produced the best results in my experimentation.
river_max = 0.05

# Normalise the tracer concentration using a power function.
# This will enhance lower concentrations.
# clip=True is required to keep values between 0 and 1 as we scaled the river max down
norm = matplotlib.colors.PowerNorm(0.3, vmin=0, vmax=river_max, clip=True)
river_tracers = river_tracers.copy(data=norm(river_tracers.values))

Assign a colour to each river, plus a colour for the ocean

In [ ]:
# Assume that anything that isn't tracer is ocean water
ocean = 1 - river_tracers.sum(dim='tracer')
ocean = xarray.where(numpy.isnan(river_tracers).any(dim='tracer'), numpy.nan, ocean)
ocean = ocean.assign_coords({'tracer': 'ocean'})
all_tracers = xarray.concat([river_tracers, ocean], dim='tracer')

# The colours for the river tracers and for ocean water.
# I've tried a few different colour schemes. This is the best I've come up with.
# Feel free to experiment!
colour_dark = [
    [0.50, 0.75, 0.00, 1.00],
    [0.75, 0.00, 0.50, 1.00],
    [0.00, 0.50, 0.75, 1.00],
    [0.00, 0.00, 0.00, 1.00],
]
colour_light = [
    [0.25, 0.50, 0.00, 1.00],
    [0.50, 0.00, 0.25, 1.00],
    [0.00, 0.25, 0.50, 1.00],
    [1.00, 1.00, 1.00, 1.00],
]
ocean_colours = xarray.DataArray(
    data=colour_light,
    dims=['tracer', 'component'],
    coords={
        'tracer': all_tracers.coords['tracer'],
        'component': ['r', 'g', 'b', 'a'],
    },
)

Plot the river tracers overlayed on MODIS true colour satellite imagery from NASA GIBS.

In [ ]:
# Set up the figure for plotting
figure = plt.figure(
    figsize=tuple(i / DPI for i in IMAGE_SIZE),
    dpi=DPI,
    layout='constrained')

axes = figure.add_subplot(projection=cartopy.crs.PlateCarree())
plt.title("River tracer concentration")
    
# Cover the map extent with the plot, expanding the extent as necessary to
# keep the image aspect ratio.
axes.set_extent(MAP_EXTENT)
axes.set_aspect('equal', adjustable='datalim')


# Add MODIS baselayer from NASA GIBS
axes.add_wms(
    wms='https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi',
    layers=['MODIS_Terra_CorrectedReflectance_TrueColor'],
    wms_kwargs={'time': MODIS_TIMESTEP},
)


# Add river tracer data
cell_colours = (all_tracers * ocean_colours).sum(dim='tracer')
cell_colours = cell_colours.clip(min=0., max=1.)
cell_colours = convention.ravel(cell_colours).transpose()[convention.mask].values
axes.add_collection(convention.make_poly_collection(
    color=cell_colours,
    edgecolor="face",
))


# Add river annotaions
bbox = dict(facecolor='white', edgecolor='black', linewidth=0.5)
for river_name in river_names:
    colour_drawing = matplotlib.offsetbox.DrawingArea(8, 8, 0, 0)
    colour_drawing.add_artist(matplotlib.patches.Rectangle(
        (0, 0), 8, 8,
        facecolor=ocean_colours.sel(tracer=river_name).values,
        edgecolor='black', linewidth=0.5,
    ))
    river_label = matplotlib.offsetbox.TextArea(river_labels[river_name], textprops=dict(color='black'))
    river_offsetbox = matplotlib.offsetbox.HPacker(
        children=[colour_drawing, river_label],
        pad=1, sep=5, align='center',
    )
    river_annotation = matplotlib.offsetbox.AnnotationBbox(
        river_offsetbox,
        xy=river_coords[river_name], xycoords='data',
        xybox=(-10, -10), boxcoords='offset points', box_alignment=(1, 1),
        bboxprops=bbox,
    )
    axes.add_artist(river_annotation)


# Add some colour bars
first = True
ocean_colour = ocean_colours.sel(tracer='ocean').values
for tracer in ocean_colours.coords['tracer']:
    if tracer == 'ocean':
        continue
    river_colour = ocean_colours.sel(tracer=tracer).values
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list(tracer, [
        [0, ocean_colour],
        [1, river_colour],
    ])
    mappable = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
    colorbar = figure.colorbar(mappable, ax=axes, pad=0, aspect=50)
    if first:
        colorbar.set_label('River tracer concentration')
        colorbar.formatter = matplotlib.ticker.FuncFormatter(lambda x, pos: f'{x:.0%}')
    else:
        colorbar.set_ticks([])

    first = False


# Add attribution statement for base layer imagery
attribution = axes.add_artist(matplotlib.offsetbox.AnnotationBbox(
    matplotlib.offsetbox.PaddedBox(
        child=matplotlib.offsetbox.TextArea(
            "Satellite imagery provided by NASA's Global Imagery Browse Services (GIBS),\npart of NASA's Earth Science Data and Information System (ESDIS).",
            textprops=dict(fontsize='x-small'),
        ),
        pad=5, draw_frame=True, patch_attrs=bbox,
    ),
    xy=(1, 0), xycoords='axes fraction',
    xybox=(0, 0), box_alignment=(1, 0), boxcoords='offset points',
    frameon=False,
))
